In [1]:
import pandas as pd

result_statics = pd.read_csv("ResultStatics.csv")
rocket = pd.read_csv("Rocket.csv")
sky_cam = pd.read_csv("OurSkyCamera.csv")
nasa = pd.read_csv("nasaCamera.csv")
ams = pd.read_csv("AmericanMeteorSociety.csv")

dfs = [result_statics, rocket, sky_cam, nasa, ams]
df_names = ["result_statics", "rocket", "sky_cam", "nasa", "ams"]

for i in range(0,5):
    print(df_names[i])
    print(len(dfs[i]))
    print(dfs[i].head)
    print()


result_statics
95
<bound method NDFrame.head of       Unnamed: 0         Event ID  Reports  \
0   For 20240727  Event 3646-2024        9   
1            NaN  Event 3690-2024        1   
2            NaN  Event 3691-2024        3   
3    Daily Break  Event 3712-2024        1   
4            NaN  Event 3714-2024        1   
..           ...              ...      ...   
90           NaN  Event 4666-2024        6   
91           NaN  Event 4667-2024        2   
92   Daily Break  Event 4695-2024        2   
93           NaN  Event 4696-2024        1   
94           NaN  Event 4697-2024        1   

   Time [UTC] (average time if multiple reports)       State  \
0                            2024-07-27 00:52 UT  CT, MA, NH   
1                            2024-07-27 01:43 UT          VA   
2                            2024-07-27 02:00 UT      TX, OK   
3                            2024-07-28 05:19 UT          MD   
4                            2024-07-28 05:20 UT          NJ   
..             

In [2]:
from datetime import datetime, date, time, timezone, timedelta
import numpy as np
import statistics as st


# Nasa: YMD 1[8:], UTC 2[8:]

nasa_filtered = nasa.iloc[8:,[1,2]]
nasa_filtered = nasa_filtered.dropna()
nasa_dt = []

for i in range(0,len(nasa_filtered)):
    i_date = nasa_filtered.iat[i,0]
    i_time = nasa_filtered.iat[i,1]
    dt_str = i_date + " " + i_time
    dt_str = dt_str[:-4]
    date_format = "%Y%m%d %H:%M:%S"
    try:
        dt = datetime.strptime(dt_str, date_format)
        nasa_dt.append(dt)
    except:
        nasa_dt.append(np.nan)

nasa_filtered['datetime'] = nasa_dt
nasa_filtered = nasa_filtered.dropna()
nasa_filtered['event'] = 'meteor'
nasa_filtered['start_time'] = nasa_filtered['datetime'] + timedelta(seconds=-1)
nasa_filtered['end_time'] = nasa_filtered['datetime'] + timedelta(seconds=1)

nasa_filtered = nasa_filtered.iloc[:, [3,4,5]]
nasa_filtered['source'] = 'NASA'
nasa_filtered = nasa_filtered.dropna()

# AMS: Full UTC DT 3[7:]
ams_dt = []
ams_filtered = ams.iloc[7:,3]
ams_filtered = ams_filtered.dropna()
ams_filtered = ams_filtered.to_frame()
ams_filtered['event'] = 'meteor'
ams_filtered.rename(columns={ams_filtered.columns[0]: 'datetime'}, inplace=True)

for i in range(0, len(ams_filtered)):
    dt_str = ams_filtered.iloc[i,0]
    dt_str = dt_str[:-3]
    date_format = "%Y-%m-%d %H:%M"
    try:
        dt = datetime.strptime(dt_str, date_format)
        ams_dt.append(dt)
    except:
        ams_dt.append(np.nan)

ams_filtered['datetime'] = ams_dt
ams_filtered['start_time'] = ams_filtered['datetime'] + timedelta(seconds=-1)
ams_filtered['end_time'] = ams_filtered['datetime'] + timedelta(seconds=1)
ams_filtered = ams_filtered.iloc[:,[1,2,3]]
ams_filtered['source'] = 'AMS'
ams_filtered = ams_filtered.dropna()

# Rocket: Longform date 2[], start time UTC 3[], end UTC 4[]
rocket['start_str'] = ['5:45:00', '5:09:00', '9:22:00', '10:45:00', '1:14:00', '5:01:00',
                       '4:39:00', '7:24:00', '3:02:53', '6:42:00', '12:50:00', '1:18:00',
                       '2:02:00', '10:37:00']
rocket['date'] = ['2024-07-27', '2024-07-28', '2024-07-28', '2024-07-30', '2024-08-01',
                  '2024-08-02', '2024-08-02', '2024-08-04', '2024-08-04', '2024-08-06',
                  '2024-08-10', '2024-08-11', '2024-08-12', '2024-08-12']

with_ends = rocket.drop([3,4,9])

with_ends['end_str'] = ['5:53:50', '5:17:50', '9:30:50', '5:09:50', '5:39:00', '7:32:50',
                        '3:20:53', '12:59:00', '2:18:00', '2:50:00', '10:46:00']

durations = []

full_rows = list(with_ends.index.values)

for i in full_rows:
    start_str = with_ends['start_str'][i]
    end_str = with_ends['end_str'][i]
    start_dt = datetime.strptime(start_str, "%H:%M:%S")
    end_dt = datetime.strptime(end_str, "%H:%M:%S")
    diff: timedelta = end_dt - start_dt
    durations.append(diff.seconds)

med_dur = st.median(durations)
rocket_starts = []
rocket_ends = []
for i in range(0, len(rocket)):
    start = rocket['date'][i] + " " + rocket['start_str'][i]
    rocket_starts.append(datetime.strptime(start, "%Y-%m-%d %H:%M:%S"))
    if i in full_rows:
        end = rocket['date'][i] + " " + with_ends['end_str'][i]
        rocket_ends.append(datetime.strptime(end, "%Y-%m-%d %H:%M:%S"))
    else:
        rocket_ends.append(rocket_starts[i] + timedelta(seconds=med_dur))

rocket['start_time'] = rocket_starts
rocket['end_time'] = rocket_ends
rocket['event'] = 'rocket'
rocket['source'] = 'Rocket'

rocket_filtered = rocket.iloc[:, [8,9,10,11]]
rocket_filtered = rocket_filtered.dropna()

# Sky_cam: Full UTC 1[]
    # Seems to be manually annotated by lab team
    # Remove 4, 8, 12, 15, 17, 20, 23, 24

# result_statics: full UTC 3[]

# rs_filtered = result_statics
# rs_filtered['event'] = ''


event_data = pd.concat([ams_filtered, nasa_filtered, rocket_filtered])
event_data.reset_index()
event_data.to_csv("event_data.csv")